<a href="https://colab.research.google.com/github/8bitpizza/yahoo-finance/blob/master/windship_scrape_1_13_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#########################################################
#  Site Scraper with Login, Using Mechanical Soup       #
#               By. Bryce Robinson                      #
#                   1/11/2020                           #
#########################################################
# About:
#
# This scraper is intended to scrape my supplier's website for Product
# Information needed for my wbsite.
#-------------------------------------------------------------------------------
# Libraries Used:
#
# mechanicalsoup - https://mechanicalsoup.readthedocs.io/en/stable/mechanicalsoup.html 
# pandas - https://pandas.pydata.org/
#-------------------------------------------------------------------------------

!pip3 install mechanicalsoup
!pip3 install pandas

In [ ]:
import mechanicalsoup
import pandas as pd

#------------------------------------------------------------------------------- Project Lists used for Database

vendor_url_list = []
brand_list = []
title_list = []
price_list = []
sku_list = []
variation_list = []
product_description_list = []
product_photos_list = []
product_videos_list = []


#------------------------------------------------------------------------------- DataFrame Builder
# For pandas header, I created dict. and linked them to the main data lists


product_scrape_dict = {
    "VENDOR URL": vendor_url_list,
    "BRAND": brand_list,
    "TITLE": title_list,
    "PRICE": price_list,
    "SKU": sku_list,
    "VARIATION": variation_list,
    "DESCRIPTION": product_description_list,
    "PRODUCT PHOTOS": product_photos_list,
    "PRODUCT VIDEOS":product_videos_list}

#-------------------------------------------------------------------------------
#------------------------         LOGIN        ---------------------------------

browser = mechanicalsoup.StatefulBrowser(
    raise_on_404=False,
    user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0")
browser.open("https://windshiptrading.com/login.php")
browser.select_form(selector='form.flex-component__mobileUp-3.login-form.login-form-account.form')
login_form = browser.select_form(selector='form.flex-component__mobileUp-3.login-form.login-form-account.form')
login_form.set_input({"login_email": "sales@samuraiblaze.com", "login_pass": "BMW01car"})
browser.submit_selected()
if browser.url == "https://windshiptrading.com/account.php?action=order_status":
    print("Logged In Successfully!")
else:
    print("Login ERROR")

#-------------------------------------------------------------------------------
#------------------------      After Login     ---------------------------------
#-------------------------------------------------------------------------------
# The following lines of code should be run after the login is successful
#------------------------------------------------------------------------------- Main Scrape Data Lists

class LinkNotFoundError(Exception):
  pass

#------------------------------------------------------------------------------- 
#------------------------   CATEGORY LINKS   -----------------------------------
#------------------------------------------------------------------------------- Category Page Link Builder
# 
# Must launch function vendor_category_links("url for first page")
#
#
vendor_category_link_list = []

def start(url):
  browser.open(url)
  vendor_category_link_list.append(browser.url)
 
  next_url_link = browser.follow_link('a', class_="pagination-link pagination-link--next")
  #browser.follow_link('a', class_="pagination-link pagination-link--next")
  vendor_category_link_list.append(browser.url)
  print(vendor_category_link_list)
  try:
    while next_url_link is not None:
      browser.follow_link('a', class_="pagination-link pagination-link--next")
      vendor_category_link_list.append(browser.url)
      print(vendor_category_link_list)
  except LinkNotFoundError(Exception):
    pass



def run_scraper():
  for url in vendor_category_link_list:
    #url = input("Enter Url to scrape after login:  ")  #<----------------      User Input for URL
     
    browser.open(url)
    product_links_list = []
    category_page = browser.get_current_page()
    product_link = category_page.find_all('a', class_="card-figure-link")
    for link in product_link:
      product_links_list.append(link.get('href'))

    for product_links in product_links_list:
      
      browser.open(product_links)
      vendor_url = browser.url
      product_page = browser.get_current_page()


      # Code is built under 'for' loop that iterates through
      # product_link_list[]

      ###-------Vendor URL---------###

      
      
      print(browser.url)
      vendor_url_list.append(browser.url)
      


      ###-----------Brand----------###
      
      brand = product_page.find('h2', class_="productView-brand")
      if brand == None:
        print("ERROR")
        brand_list.append("ERROR")
    #    print(brand_list)#<----Debug
      
      else:
        print(brand.text.strip())
        brand_list.append(brand.text.strip())
    #    print(brand_list)#<----Debug

      ###-----------Title----------###
      

      title = product_page.find('h1', class_="productView-title")
      if title == None:
        print("ERROR")
        title_list.append("ERROR")
    #    print(title_list)#<----Debug
      
      else:
        print(title.text.strip())
        title_list.append(title.text.strip())
    #    print(title_list)#<----Debug



      ###-----------Price----------###
      

      price = product_page.find('span', class_="price price--withoutTax price-primary")
      if price == None:
        print("ERROR")
        price_list.append("ERROR")
    #    print(price_list)#<----Debug
      
      else:
        print(price.text.strip())
        price_list.append(price.text.strip())
    #    print(price_list)#<----Debug  

    
      ###-----------Sku----------###



      sku = product_page.find('dd', class_="productView-info-value productView-info-value--sku")
      
      if sku == None:
        print("ERROR")
        sku_list.append("ERROR")
    #    print(sku_list)
      
      else:
        print(sku.text.strip())
        sku_list.append(sku.text.strip())
    #    print(sku_list)



      
      
      ###-----------Variation----------### 
      
      variation = product_page.find('select', class_="form-select form-select--small")
      if variation == None:
        variation_list.append("None")
        
        print("n/a")
      #    print(variation_list)

      else:
        print(variation.text.replace("Choose Options","").strip())
        variation_list.append(variation.text.replace("Choose Options","").strip())


      
      ###------ Product Description -----###

      product_html = product_page.find("article", id="productDescription") 
      if product_html == None:
        product_description_list.append("Description is missing, please email sales@samuraiblaze.com for product question")
        
        print("Description is missing, please email sales@samuraiblaze.com for product questions")
    #    print(product_description_list)

      else:

        if "Wholesale" in product_html.text:
          print(product_html.text.replace("Wholesale ", '').strip("/n"))
          product_description_list.append(product_html.text.replace("Wholesale ", '').strip("/n"))
        else:
          print(product_html.text.strip("/n"))
          product_description_list.append(product_html.text.strip("/n"))



      ###------ Product Photos ----------###
      
      multi_image_list=[]

      images = product_page.find('div', class_="sticky-product")
      additional_photos = images.find_all("a")

      if images == None:
        
        product_photos_list.append("ERROR")
        
        print("ERROR")
    #    print(product_photos_list)
      
      else:  
        for links in additional_photos:
          product_images = links.get('href')
        
          multi_image_list.append(product_images)


        print(multi_image_list)
        product_photos_list.append(multi_image_list)
      
        
      
    #      print(product_photos_list)


      ###----- Product Videos ----------###
      
      
      videos = product_page.find("iframe", id="player")

      if videos == None:
        print("None")
        product_videos_list.append("None")
    #    print(product_videos_list)
      else:
        print(videos)
        product_videos_list.append(videos)
    #    print(product_videos_list)


      ###------- Related Products -----###

    #  related_products = product_page.find("div", class_="slick-track")
    #  if related_products == None:
    #    print("None")
    #  else:
    #    for i in related_products:
    #      related_product_sku = related_products.find("sku")
    #      print(related_products_sku.text)



# Matching Shopify Input Fields

#------------------------------------------------------------------------------- Creates Data Frame per Shopify



shopify_output = {
    "VENDOR URL": vendor_url_list,
    "Title": title_list,
    "Body (HTML)": product_description_list,
    "Vendor": brand_list,
    "Variant SKU": sku_list,
    "Option1 Value": variation_list,
    "Variant Cost": price_list,
    "Image Src": product_photos_list,
    "PRODUCT VIDEOS":product_videos_list}
    

start('https://windshiptrading.com/browse-everything/')
run_scraper()
df = pd.DataFrame(shopify_output)
df


Logged In Successfully!
['https://windshiptrading.com/browse-everything/', 'https://windshiptrading.com/browse-everything/?sort=newest&page=2']
['https://windshiptrading.com/browse-everything/', 'https://windshiptrading.com/browse-everything/?sort=newest&page=2', 'https://windshiptrading.com/browse-everything/?sort=newest&page=3']
['https://windshiptrading.com/browse-everything/', 'https://windshiptrading.com/browse-everything/?sort=newest&page=2', 'https://windshiptrading.com/browse-everything/?sort=newest&page=3', 'https://windshiptrading.com/browse-everything/?sort=newest&page=4']
['https://windshiptrading.com/browse-everything/', 'https://windshiptrading.com/browse-everything/?sort=newest&page=2', 'https://windshiptrading.com/browse-everything/?sort=newest&page=3', 'https://windshiptrading.com/browse-everything/?sort=newest&page=4', 'https://windshiptrading.com/browse-everything/?sort=newest&page=5']
['https://windshiptrading.com/browse-everything/', 'https://windshiptrading.com/br

TypeError: ignored

In [ ]:
start('https://windshiptrading.com/browse-everything/')


['https://windshiptrading.com/browse-everything/', 'https://windshiptrading.com/browse-everything/?sort=newest&page=2', 'https://windshiptrading.com/browse-everything/?sort=newest&page=3', 'https://windshiptrading.com/browse-everything/?sort=newest&page=4', 'https://windshiptrading.com/browse-everything/?sort=newest&page=5', 'https://windshiptrading.com/browse-everything/?sort=newest&page=6', 'https://windshiptrading.com/browse-everything/?sort=newest&page=7', 'https://windshiptrading.com/browse-everything/?sort=newest&page=8', 'https://windshiptrading.com/browse-everything/?sort=newest&page=9', 'https://windshiptrading.com/browse-everything/?sort=newest&page=10', 'https://windshiptrading.com/browse-everything/?sort=newest&page=11', 'https://windshiptrading.com/browse-everything/?sort=newest&page=12', 'https://windshiptrading.com/browse-everything/?sort=newest&page=13', 'https://windshiptrading.com/browse-everything/?sort=newest&page=14', 'https://windshiptrading.com/browse-everything/

TypeError: ignored

In [ ]:
run_scraper()


Streaming output truncated to the last 5000 lines.
https://windshiptrading.com/linx-hypnos-zero-atomizer/
Linx
Linx Hypnos Zero Atomizer
$16.00
VAC-124-01
n/a

Description

 


Extra Information


Brand:
Linx



['https://cdn11.bigcommerce.com/s-b53sbuhr/images/stencil/1280x1280/products/4218/11288/linx-hypnos-zero-wax-vaporizer-atomizer__37914.1474391128.png?c=2']
None
https://windshiptrading.com/linx-hypnos-original-vaporizer/
Linx
Linx Hypnos Original Vaporizer
$18.50
VAP-124-02
n/a

Description

 
 


Extra Information


Brand:
Linx



['https://cdn11.bigcommerce.com/s-b53sbuhr/images/stencil/1280x1280/products/4217/11289/linx-hypnos-portable-vaporizer-front-1__52969.1496343331.png?c=2']
None
https://windshiptrading.com/atmos-kiln-ra-kit-white/
Atmos
Atmos Kiln RA Kit - White
$40.00
VAP-100-11-W
n/a

Description

Wholesale Atmos Kiln RA Kit - White
The Atmos Kiln RA builds upon the original Kiln attachment by introducing revolutionary rebuildable atomizers to the iconic waxy heatin

In [ ]:
df =pd.DataFrame(shopify_output)
df.to_csv("/content/drive/MyDrive/WINDSHIP TRADING PRODUCTS/windship_database.csv")


In [ ]:
#---------------------- DEBUG FUNCTIONS-----------------------------------------
#------------------------------------------------------------------------------- Clear data frame
def clear_data_frame():
  del product_scrape_dict

#------------------------------------------------------------------------------- Debug ValueError: arrays must all be same length
### Debug error: ValueError: arrays must all be same length
### Error is raised when Dataframe dict. are not the same length
### 
### Since the Dataframe is built off the lists created, I can debug the problem
### by viewing the len() of the list item
### 
### In this case,  my product_photo_list is greater than the other lists.
### Possible solution: List merge the output to a single lie string


def debug_list_length():

  print(
      len(vendor_url_list),
      len(brand_list),
      len(title_list),
      len(price_list),
      len(sku_list),
      len(variation_list),
      len(product_description_list),
      len(product_photos_list),
      len(product_videos_list),
  )

In [ ]:
df = pd.DataBase(shopify_output)

AttributeError: ignored